## Atlas Cohort Exchange - Results Import
This notebook is meant for importing the cohort inclusion results from an external Atlas site into the central Atlas site.

There are several steps in this process:
1. Export cohort definition as a JSON file (_see Atlas Cohort Exchange - Admin.docx_)
2. Create the same cohort definition at remote Atlas site, create inclusion report (_see Atlas Cohort Exchange - User Guide.docx_)
3. Export cohort inclusion report results to JSON file using the **AtlasCohortExchange R** package (_see Atlas Cohort Exchange - User Guide.docx_)
4. Import cohort inclusion report results into central Atlas instance using (this) Jupyter notebook (_see Atlas Cohort Exchange - Admin.docx_)

## Parameters - check/modify before running rest of notebook
These are the parameters to be used when connecting to the central Atlas database to upload the results.

###### Database parameters
The parameters needed for connecting to the relevant database and (webapi) schema representing the remote Atlas site.

###### Path to the JSON file
The path to the JSON file with the cohort inclusion report results from the remote Atlas site.

###### Cohort definition ID
The cohort_definition_id of the central Atlas instance for this cohort. Note that the Cohort ID for this cohort definition most likely is different at the remote Atlas instance from the one at the central Atlas instance. It is therefore important to here specify the Cohort ID of the Cohort definition in the **central** Atlas instance, so that the results from the remote Atlas instance can correctly be tied to the orginiating Cohort definition.

In [1]:
# ##### Database parameters ##### 
dbDbms = 'postgresql' #  ONLY POSTGRESQL SUPPORTED CURRENTLY
dbAddress = '34.251.82.109'
dbDatabase = 'testx'
dbSchema = 'atlas'
dbUser = 'username'
dbPassword='password'

# ##### Path to the JSON file ##### 
jsonFilePath = 'C:/dwn/hypertension_results.json'

# ##### Cohort definition ID ##### 
cohortId=147

## Code - load the results
Once the parameters above have been checked/modified, please run the remainder of the notebook cells to load the cohort inlusion report results into the central Atlas database.

In [2]:
# Import necessary libraries
import pandas as pd
import psycopg2
import pymssql
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [3]:
# Create connection string - ONLY POSTGRESQL SUPPORTED CURRENTLY
if dbDbms == 'postgresql':
    dbConnectionString = 'postgresql+psycopg2://' + dbUser + ':' + dbPassword + '@' + dbAddress + '/' + dbDatabase
else:
    dbConnectionString = ''

In [4]:
# Create the canvas for a map between Python objects and database records
Base = automap_base()

# Identify database, create the map from the schema, and prepare to create sessions
engine = create_engine(dbConnectionString)
Base.prepare(engine, reflect=True)
Session = sessionmaker(bind=engine)

In [5]:
# load the JSON file
with open(jsonFilePath) as data_file:    
    data = json.load(data_file)

In [7]:
# cohort
co = pd.DataFrame(data['cohort'])
co.id=cohortId
co.to_sql('cohort', engine, schema=dbSchema, if_exists='replace', index=False)

In [8]:
# cohort_inclusion
ci = pd.DataFrame(data['cohort_inclusion'])
ci.cohort_definition_id=cohortId
ci.to_sql('cohort_inclusion', engine, schema=dbSchema, if_exists='replace', index=False)

In [9]:
# cohort_inclusion_result
cir = pd.DataFrame(data['cohort_inclusion_result'])
cir.cohort_definition_id=cohortId
cir.to_sql('cohort_inclusion_result', engine, schema=dbSchema, if_exists='replace', index=False)

In [10]:
# cohort_inclusion_stats
cis = pd.DataFrame(data['cohort_inclusion_stats'])
cis.cohort_definition_id=cohortId
cis.to_sql('cohort_inclusion_stats', engine, schema=dbSchema, if_exists='replace', index=False)

In [11]:
# cohort_summary_stats
css = pd.DataFrame(data['cohort_summary_stats'])
css.cohort_definition_id=cohortId
css.to_sql('cohort_summary_stats', engine, schema=dbSchema, if_exists='replace', index=False)